In [5]:
import os
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import math


dataset_dir = 'PlantVillage'


image_paths = []
labels = []
for class_name in os.listdir(dataset_dir):
    class_dir = os.path.join(dataset_dir, class_name)
    for image_name in os.listdir(class_dir):
        image_path = os.path.join(class_dir, image_name)
        image_paths.append(image_path)
        labels.append(class_name)


label_to_index = {label: index for index, label in enumerate(sorted(set(labels)))}
labels = [label_to_index[label] for label in labels]
labels = to_categorical(labels)


train_image_paths, val_image_paths, train_labels, val_labels = train_test_split(
    image_paths, labels, test_size=0.2, random_state=42
)


image_size = (128, 128)
batch_size = 32

train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

train_generator = train_datagen.flow_from_directory(
    dataset_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training',
    shuffle=True
)

val_datagen = ImageDataGenerator(rescale=1./255)

val_generator = val_datagen.flow_from_directory(
    dataset_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation',
    shuffle=True
)


num_classes = len(label_to_index)  
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))  


model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


epochs = 10
steps_per_epoch = train_generator.n // train_generator.batch_size
validation_steps = math.ceil(val_generator.n / val_generator.batch_size)  

model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=epochs,
    validation_data=val_generator,
    validation_steps=validation_steps
)


model.save('tomato_disease_model.h5')


Found 20638 images belonging to 15 classes.
Found 0 images belonging to 15 classes.
Epoch 1/10
644/644 [==============================] - 782s 1s/step - loss: 1.7562 - accuracy: 0.4378
Epoch 2/10
644/644 [==============================] - 426s 661ms/step - loss: 1.0682 - accuracy: 0.6471
Epoch 3/10
644/644 [==============================] - 427s 663ms/step - loss: 0.8790 - accuracy: 0.7081
Epoch 4/10
644/644 [==============================] - 424s 658ms/step - loss: 0.7497 - accuracy: 0.7454
Epoch 5/10
644/644 [==============================] - 447s 694ms/step - loss: 0.6447 - accuracy: 0.7863
Epoch 6/10
644/644 [==============================] - 435s 675ms/step - loss: 0.5751 - accuracy: 0.8106
Epoch 7/10
644/644 [==============================] - 427s 662ms/step - loss: 0.5224 - accuracy: 0.8249
Epoch 8/10
644/644 [==============================] - 419s 650ms/step - loss: 0.4735 - accuracy: 0.8397
Epoch 9/10
644/644 [==============================] - 420s 652ms/step - loss: 0.4476 - 

C:\Users\User\anaconda3\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [13]:

import cv2
import numpy as np
from tensorflow.keras.models import load_model


model = load_model('tomato_disease_model.h5')


disease_classes = {
    0: "Bacterial Spot",
    1: "Early Blight",
    2: "Late Blight",
    3: "Leaf Mold",
    4: "Septoria Leaf Spot",
    5: "Spider Mites",
    6: "Target Spot",
    7: "Yellow Leaf Curl Virus",
    8: "Mosaic Virus",
    9: "Healthy"
}


def preprocess_image(image_path):
    image = cv2.imread(image_path)
    image = cv2.resize(image, (128, 128))
    image = image.astype("float") / 255.0
    image = np.expand_dims(image, axis=0)
    return image


def classify_disease(image_path):
    preprocessed_image = preprocess_image(image_path)
    predictions = model.predict(preprocessed_image)
    predicted_class = np.argmax(predictions)
    disease = disease_classes[predicted_class]
    confidence = predictions[0][predicted_class] * 100
    return disease, confidence

# Example usage
#image_path = 'tomato.jpg'
#disease, confidence = classify_disease(image_path)
#print("Detected Disease:", disease)
#print("Confidence:", confidence)


1/1 [==============================] - 0s 108ms/step
Detected Disease: Target Spot
Confidence: 46.46153151988983


In [29]:
from flask import Flask, render_template, request
import cv2
import numpy as np
from tensorflow.keras.models import load_model

app = Flask(__name__)


model = load_model('tomato_disease_model.h5')


disease_classes = {
    0: "Bacterial Spot",
    1: "Early Blight",
    2: "Late Blight",
    3: "Leaf Mold",
    4: "Septoria Leaf Spot",
    5: "Spider Mites",
    6: "Target Spot",
    7: "Yellow Leaf Curl Virus",
    8: "Mosaic Virus",
    9: "Healthy"
}


def preprocess_image(image_path):
    image = cv2.imread(image_path)
    image = cv2.resize(image, (128, 128))
    image = image.astype("float") / 255.0
    image = np.expand_dims(image, axis=0)
    return image


def classify_disease(image_path):
    preprocessed_image = preprocess_image(image_path)
    predictions = model.predict(preprocessed_image)
    predicted_class = np.argmax(predictions)
    disease = disease_classes[predicted_class]
    confidence = predictions[0][predicted_class] * 100
    return disease, confidence

@app.route('/', methods=['GET', 'POST'])
def index():
    if request.method == 'POST':
        
        file = request.files['file']
        image_path = 'static/uploads/image.jpg'
        file.save(image_path)

        
        disease, confidence = classify_disease(image_path)

        return render_template('result.html', disease=disease, confidence=confidence)
    return render_template('index.html')

if __name__ == '__main__':
    app.run(debug=True, use_reloader=False)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
